# Generate graphRAG for Financial Data #

## Imports ##

In [1]:
import os
import glob
import time
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
from typing import List

from langchain_community.document_loaders import TextLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_text_splitters import TokenTextSplitter
from neo4j.exceptions import ClientError

import openai
from openai import OpenAI
from openai import AzureOpenAI

from time import sleep



## Initializations ##

In [2]:
# Load from environment
load_dotenv('.env', override=True)

AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY"), 



# Embeddings & LLM models
embedding_dimension = 1536
embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3",api_version="2024-02-01",dimensions=embedding_dimension)

llm = AzureChatOpenAI(azure_deployment='chat_gtp_35',api_version="2023-05-15", temperature=0)

# Get Neo4j credentials from environment variables
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")

graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
sleep(5)
graph.query("MATCH (n) DETACH DELETE n")

class Questions(BaseModel):
    """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on " "the information from the text"
        ),
    )


questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating a maximum of 10 hypothetical questions based on the information and return the output in JSON format "
                "Make sure to provide full context in the generated "
                "questions but again only 10 questions are required."
            ),
        ),
        (
            "human",
            (
                "Use the given format to generate hypothetical questions from the"
                "following input: {input}"
            ),
        ),
    ]
)

question_chain = questions_prompt | llm.with_structured_output(Questions.json)

# Ingest summaries

summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating concise and accurate summaries based on the "
                "information found in the text."
            ),
        ),
        (
            "human",
            ("Generate a summary of the following input: {question}\n" "Summary:"),
        ),
    ]
)

summary_chain = summary_prompt | llm

def clean_questions_data(questions_data):
    if 'questions' not in questions_data:
        return []

    cleaned_questions = []
    for entry in questions_data['questions']:
        if isinstance(entry, dict) and 'question' in entry:
            cleaned_questions.append(entry)
        elif isinstance(entry, str):
            cleaned_questions.append({'question': entry})
        else:
            # Handle cases where the entry is not a string or doesn't contain the 'question' key
            print(f"Invalid entry found and skipped: {entry}")
    # Convert the cleaned_questions list into a pandas DataFrame
    df = pd.DataFrame(cleaned_questions)

    # Generate a unique file ID using the current timestamp
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    file_id = f"questions_{timestamp}"

    # Save the DataFrame into a CSV file with the unique file ID
    df.to_csv(f"./LLM_questions/{file_id}.csv", index=False)
    return cleaned_questions

## Data ##

In [3]:
# import getFinancialInfo3

# com_list = pd.read_csv('ticker_select_test.txt', sep='\t',names=['symbol', 'cik'])
# data = getFinancialInfo_3.get_data(com_list,True)
# data

,cik,name,entityType,sic,sicDescription,tickers,exchanges,category,phone,address,10-K-1
0,789019,MICROSOFT CORP,operating,7372,Services-Prepackaged Software,[MSFT],[Nasdaq],Large accelerated filer,425-882-8080,ONE MICROSOFT WAY REDMOND WA 98052-6399,"[0000950170-23-035122, [[10_K_MSFT_0000950170-..."
1,320193,Apple Inc.,operating,3571,Electronic Computers,[AAPL],[Nasdaq],Large accelerated filer,(408) 996-1010,ONE APPLE PARK WAY CUPERTINO CA 95014,"[0000320193-23-000106, [[10_K_AAPL_0000320193-..."
2,1045810,NVIDIA CORP,operating,3674,Semiconductors & Related Devices,[NVDA],[Nasdaq],Large accelerated filer,408-486-2000,2788 SAN TOMAS EXPRESSWAY SANTA CLARA CA 95051,"[0001045810-24-000029, [[10_K_NVDA_0001045810-..."


In [ ]:
# pip install sec-api
# from sec_api import ExtractorApi
# import multiprocessing

# API_KEY = =os.getenv("SEC_API")

# extractorApi = ExtractorApi(API_KEY)

# def extract_items_10k(filing_url):
#   items = ["1", "1A", "7",
#            "7A"]
#   filing_name = os.path.basename(filing_url)

#   for item in items:
#     print("item:", item, "url", filing_url)

#     try:
#       section_text = extractorApi.get_section(filing_url=filing_url,
#                                               section=item,
#                                               return_type="text")

#       # do something with section_text. for example, save to disk, in a database
#       # or perform analytics
#       # IMPORTANT: you don't want to hold a large number of sections in memory
#       # and add sections to a list. otherwise you end up with out-of-memory issues.
#       # instead make sure to let the garbage collection release memory frequently.

#       # Create a new directory for the current filing if it doesn't exist
#       if not os.path.exists(filing_name):
#         os.makedirs(filing_name)

#       # Save the extracted text to a file
#       with open(f"{filing_name}/{item}.txt", "w") as f:
#         f.write(section_text)
#     except Exception as e:
#       print(e)

# urls_10k = ["https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm"]

# # represents the number of processes to run in parallel.
# # if you perform a CPU-light task such as saving a section to your local disk,
# # you might want to set the number of processes slighly higher
# # than number of CPU cores. if you perform CPU-heavy tasks, set number_of_processes
# # to the actual number of CPU cores.
# number_of_processes = 2

# with multiprocessing.Pool(number_of_processes) as pool:
#   pool.map(extract_items_10k, urls_10k)

## Code ##

In [21]:

def extract_entities_relationships(folder):
    
    files = glob.glob(f'./sec_10K_data/{folder}/*.txt')
    start = time.perf_counter()  # Assign a default value to the start variable
    print(f"Running pipeline for {len(files)} files in {folder} folder")

    # Iterate over the list of file paths and print each one
    for file_path in files:
        print(f"Extracting entities and relationships for: {str(file_path)}")
    
        # Load the text file
        #loader = TextLoader(str(txt_path))
        loader = TextLoader(str(file_path), encoding='utf-8')
        documents = loader.load()
        # print(documents)
       
        # Ingest Parent-Child node pairs
        parent_splitter = TokenTextSplitter(chunk_size=512*5, chunk_overlap=24)
        child_splitter = TokenTextSplitter(chunk_size=100*5, chunk_overlap=24)
        parent_documents = parent_splitter.split_documents(documents)
        
        for i, parent in enumerate(parent_documents):
            child_documents = child_splitter.split_documents([parent])
            params = {
                "parent_text": parent.page_content,
                "parent_id": i,
                "parent_embedding": embeddings.embed_query(parent.page_content),
                "children": [
                    {
                        "text": c.page_content,
                        "id": f"{i}-{ic}",
                        "embedding": embeddings.embed_query(c.page_content),
                    }
                    for ic, c in enumerate(child_documents)
                ],
            }
            # Ingest data
            graph.query(
                """
            MERGE (p:Parent {id: $parent_id})
            SET p.text = $parent_text
            WITH p
            CALL db.create.setVectorProperty(p, 'embedding', $parent_embedding)
            YIELD node
            WITH p 
            UNWIND $children AS child
            MERGE (c:Child {id: child.id})
            SET c.text = child.text
            MERGE (c)<-[:HAS_CHILD]-(p)
            WITH c, child
            CALL db.create.setVectorProperty(c, 'embedding', child.embedding)
            YIELD node
            RETURN count(*)
            """,
                params,
            )
            # Create vector index for child
            try:
                graph.query(
                    "CALL db.index.vector.createNodeIndex('parent_document', "
                    "'Child', 'embedding', $dimension, 'cosine')",
                    {"dimension": embedding_dimension},
                )
            except ClientError:  # already exists
                pass
            # Create vector index for parents
            try:
                graph.query(
                    "CALL db.index.vector.createNodeIndex('typical_rag', "
                    "'Parent', 'embedding', $dimension, 'cosine')",
                    {"dimension": embedding_dimension},
                )
            except ClientError:  # already exists
                pass

        
        
        for i, parent in enumerate(parent_documents):

            questions_data = question_chain.invoke(parent.page_content)
            cleaned_questions_list = clean_questions_data(questions_data) 
            # print(cleaned_questions_list)
            if 'questions' in questions_data:
                questions_list = questions_data['questions']  # Access the 'questions' list
                params = {
                    "parent_id": i,
                    "questions": []
                }

            for iq, q in enumerate(cleaned_questions_list):  # Use the cleaned questions list
                if isinstance(q, dict) and 'question' in q:
                    try:
                        question_text = q['question']
                        embedding = embeddings.embed_query(question_text)
                        params["questions"].append({"text": question_text, "id": f"{i}-{iq}", "embedding": embedding})
                    except Exception as e:
                        print(f"Error embedding question {q}: {e}")
                else:
                    print(f"Skipping invalid question entry: {q}")

                sleep(5)
                graph.query(
                    """
                MERGE (p:Parent {id: $parent_id})
                WITH p
                UNWIND $questions AS question
                CREATE (q:Question {id: question.id})
                SET q.text = question.text
                MERGE (q)<-[:HAS_QUESTION]-(p)
                WITH q, question
                CALL db.create.setVectorProperty(q, 'embedding', question.embedding)
                YIELD node
                RETURN count(*)
                """,
                    params,
                )
                # Create vector index
                try:
                    graph.query(
                        "CALL db.index.vector.createNodeIndex('hypothetical_questions', "
                        "'Question', 'embedding', $dimension, 'cosine')",
                        {"dimension": embedding_dimension},
                    )
                except ClientError:  # already exists
                    pass
                
        for i, parent in enumerate(parent_documents):
            summary = summary_chain.invoke({"question": parent.page_content}).content
            params = {
                "parent_id": i,
                "summary": summary,
                "embedding": embeddings.embed_query(summary),
            }
            graph.query(
                """
            MERGE (p:Parent {id: $parent_id})
            MERGE (p)-[:HAS_SUMMARY]->(s:Summary)
            SET s.text = $summary
            WITH s
            CALL db.create.setVectorProperty(s, 'embedding', $embedding)
            YIELD node
            RETURN count(*)
            """,
                params,
            )
            # Create vector index
            try:
                graph.query(
                    "CALL db.index.vector.createNodeIndex('summary', "
                    "'Summary', 'embedding', $dimension, 'cosine')",
                    {"dimension": embedding_dimension},
                )
            except ClientError:  # already exists
                pass
            
    end = time.perf_counter()
    print(f"Pipeline completed in {end-start} seconds")
    
def ingestion_pipeline(folders):

     for folder in folders:
        extract_entities_relationships(folder)


## Test Data Ingestion ##

In [25]:
graph.query("""
  SHOW VECTOR INDEXES
  """
)

[]

In [26]:
folders = ["aapl-20230930"]
ingestion_pipeline(folders)

Running pipeline for 4 files in aapl-20230930 folder
Extracting entities and relationships for: ./sec_10K_data/aapl-20230930\1.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20230930\1A.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20230930\7.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20230930\7A.txt
Pipeline completed in 509.1743317999935 seconds


## Test RAG ##

In [27]:
from langchain_community.vectorstores import Neo4jVector


# Typical RAG retriever

typical_rag = Neo4jVector.from_existing_index(
    embeddings, index_name="typical_rag"
)

# Parent retriever

parent_query = """
MATCH (node)<-[:HAS_CHILD]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata LIMIT 1
"""

parent_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="parent_document",
    retrieval_query=parent_query,
)

# Hypothetic questions retriever

hypothetic_question_query = """
MATCH (node)<-[:HAS_QUESTION]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

hypothetic_question_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="hypothetical_questions",
    retrieval_query=hypothetic_question_query,
)
# Summary retriever

summary_query = """
MATCH (node)<-[:HAS_SUMMARY]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

summary_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="summary",
    retrieval_query=summary_query,
)

In [28]:
response = typical_rag.similarity_search(
    "What are some risk factors that can affect Appel's stock price?"
)
print(response[0].page_content)

 of tax laws may be uncertain, require significant judgment and be subject to differing interpretations. 

The Company is also subject to the examination of its tax returns and other tax matters by the U.S. Internal Revenue Service and other tax authorities and governmental bodies. The Company regularly assesses the likelihood of an adverse outcome resulting from these examinations to determine the adequacy of its provision for taxes. There can be no assurance as to the outcome of these examinations. If the Company&#8217;s effective tax rates were to increase, or if the ultimate determination of the Company&#8217;s taxes owed is for an amount in excess of amounts previously accrued, the Company&#8217;s business, results of operations and financial condition could be materially adversely affected. 

General Risks 

The price of the Company&#8217;s stock is subject to volatility. 

The Company&#8217;s stock has experienced substantial price volatility in the past and may continue to do s

In [29]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

vector_typrag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=typical_rag.as_retriever()
)

vector_parent = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=parent_vectorstore.as_retriever()
)

vector_hypquestion = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=hypothetic_question_vectorstore.as_retriever()
)

vector_summary = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=summary_vectorstore.as_retriever()
)

vector_typrag.invoke(
     "What are some risk factors that can affect Appel's stock price?"
)

{'query': "What are some risk factors that can affect Appel's stock price?",
 'result': "There are several risk factors that can affect Apple's stock price, including:\n\n1. Volatility in the stock market and the technology industry as a whole.\n2. Uncertainty and potential adverse outcomes resulting from tax laws and examinations by tax authorities.\n3. Economic risk from interest rates and foreign exchange rates.\n4. Legal and regulatory compliance risks, including unfavorable results of legal proceedings or government investigations.\n5. Investment in new business strategies and acquisitions that could disrupt the Company's ongoing business and present risks not originally contemplated.\n6. Risks associated with the Company's retail stores, including managing costs associated with retail store construction and operation, managing relationships with existing retail partners, and obtaining and renewing leases in quality retail locations at a reasonable cost.\n7. Risks associated with 

In [31]:
vector_hypquestion.invoke(
     "What are some risk factors that can affect Appel's stock price?"
)

{'query': "What are some risk factors that can affect Appel's stock price?",
 'result': "The given context does not provide information about all the risk factors that can affect Apple's stock price. However, the context does mention two types of risks that the company is exposed to: interest rate risk and foreign exchange rate risk. The company uses various strategies to manage these risks, but they may still impact the company's consolidated financial statements. Increases in interest rates will negatively affect the fair value of the company's investment portfolio and increase the interest expense on the company's term debt. Changes in exchange rates, and in particular a strengthening of the U.S. dollar, will negatively affect the company's net sales and gross margins as expressed in U.S. dollars. Fluctuations in exchange rates may also affect the fair values of certain of the company's assets and liabilities."}

In [32]:
vector_parent.invoke(
     "What are some risk factors that can affect Appel's stock price?"
)

{'query': "What are some risk factors that can affect Appel's stock price?",
 'result': "Some risk factors that can affect Apple's stock price include volatility in the stock market, extreme stock price and volume fluctuations, uncertainty and differing interpretations of tax laws, and the examination of the company's tax returns by tax authorities. Additionally, if the company fails to meet expectations related to future growth, profitability, dividends, share repurchases, or other market expectations, the price of the company's stock may decline significantly, which could have a material adverse impact on investor confidence and employee retention."}

In [33]:
vector_summary.invoke(
     "What are some risk factors that can affect Appel's stock price?"
)

{'query': "What are some risk factors that can affect Appel's stock price?",
 'result': "There are several risk factors that can affect Apple's stock price, including:\n\n1. Fluctuations in interest rates and foreign exchange rates\n2. Changes in consumer demand for Apple's products and services\n3. Intense competition in the technology industry\n4. Changes in laws and regulations worldwide, including antitrust, privacy, and data security laws\n5. Litigation and government investigations, which can result in legal proceedings and claims against the company\n6. Investment in new business strategies and acquisitions, which could disrupt the company's ongoing business and present risks not originally contemplated\n7. Financial risks, including fluctuations in net sales and profit margins, and exposure to credit risk and fluctuations in the values of its investment portfolio\n8. Environmental, social, and governance considerations and related reporting obligations, which expose the company

##RAG Agent Tool (combination of retrievers)

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType


cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = AzureChatOpenAI(azure_deployment='Chat_gpt_4',api_version="2023-05-15", temperature=0),
    qa_llm = AzureChatOpenAI(azure_deployment='chat_gtp_35',api_version="2023-05-15", temperature=0), graph=graph, verbose=True,
)

tools = [
    Tool(
        name="Tasks",
        func=vector_typrag.run,
        description="""Useful to answer most of the questions.
        Not useful for questions that involve aggregation.
        Use full question as input.
        """,
        
    ),
    Tool(
        name="Tasks",
        func=vector_hypquestion.run,
        description="""Useful to answer questions on dates and relationship between different companies.
        Not useful for questions that involve aggregation.
        Use full question as input.
        """,        
        
    ),
    # Tool(
    #     name="Graph",
    #     func=cypher_chain.run,
    #     description=""" Only useful for AGGREGATION questions.
    #     Use full question as input.
    #     """,
    # ),
]

mrkl = initialize_agent(
    tools, 
    AzureChatOpenAI(azure_deployment='Chat_gpt_4',api_version="2023-05-15", temperature=0),
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)


In [ ]:
graph.refresh_schema()
response = mrkl.invoke("What are some risk factors that can affect Appel's stock price?")
print(response)